<a href="https://colab.research.google.com/github/datavistics/flan-t5-e2e-qg/blob/main/flan-t5-samsum-summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Important
This work was **heavily** influenced by [this blog](https://www.philschmid.de/fine-tune-flan-t5) from Phil Schmidd. Most of the content is his, I only adjusted it for question generation. Please check his site for more great tutorials.

# Fine-tune FLAN-T5 for chat & dialogue summarization

In this blog, you will learn how to fine-tune [google/flan-t5-xl](https://huggingface.co/google/flan-t5-xl) for question generation using Hugging Face Transformers. If you already know T5, FLAN-T5 is just better at everything. For the same number of parameters, these models have been fine-tuned on more than 1000 additional tasks covering also more languages. 

In this example we will use the [Squad v1.1 t5 for Question Generation](https://huggingface.co/datasets/derek-thomas/squad-v1.1-t5-question-generation) dataset a collection of about 18k context and question set pairs. These were converted from SQuAD V1.1.

You will learn how to:

1. [Setup Development Environment](#1-setup-development-environment)
2. [Load and prepare dataset](#2-load-and-prepare-dataset)
3. [Fine-tune and evaluate FLAN-T5](#3-fine-tune-and-evaluate-flan-t5)
4. [Run Inference](#4-run-inference)

Before we can start, make sure you have a [Hugging Face Account](https://huggingface.co/join) to save artifacts and experiments. 

## Quick intro: FLAN-T5, just a better T5

FLAN-T5 released with the [Scaling Instruction-Finetuned Language Models](https://arxiv.org/pdf/2210.11416.pdf) paper is an enhanced version of T5 that has been finetuned in a mixture of tasks. The paper explores instruction finetuning with a particular focus on (1) scaling the number of tasks, (2) scaling the model size, and (3) finetuning on chain-of-thought data. The paper discovers that overall instruction finetuning is a general method for improving the performance and usability of pretrained language models. 

![flan-t5](https://www.philschmid.de/_next/image?url=%2Fstatic%2Fblog%2Ffine-tune-flan-t5%2Fflan-t5.png&w=1920&q=75)

* Paper: https://arxiv.org/abs/2210.11416
* Official repo: https://github.com/google-research/t5x

--- 

Now we know what FLAN-T5 is, let's get started. 🚀

## 1. Setup Development Environment

Our first step is to install the Hugging Face Libraries, including transformers and datasets. Running the following cell will install all the required packages. 

In [1]:
# python
%pip install transformers datasets rouge-score nltk tensorboard evaluate py7zr --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.0 MB

In [2]:
# install git-fls for pushing model and logs to the hugging face hub
!sudo apt-get install git-lfs --yes

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 27 not upgraded.


Adding device so we can use cuda.

In [3]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.backends.cuda.matmul.allow_tf32 = True

This example will use the [Hugging Face Hub](https://huggingface.co/models) as a remote model versioning service. To be able to push our model to the Hub, you need to register on the [Hugging Face](https://huggingface.co/join). 
If you already have an account, you can skip this step. 
After you have an account, we will use the `notebook_login` util from the `huggingface_hub` package to log into our account and store our token (access key) on the disk. 

In [4]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## 2. Load and prepare dataset

we will use the [Squad v1.1 t5 for Question Generation](https://huggingface.co/datasets/derek-thomas/squad-v1.1-t5-question-generation) dataset a collection of about 18k context and question set pairs. These were converted from SQuAD V1.1. The version below is truncated for brevity.

```json
{
  "summary": "generate questions: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary...is a simple, modern stone statue of Mary.",
  "dialogue": "To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France? {sep_token} What...Building at Notre Dame? {sep_token}"
}
```

In [5]:
dataset_id = "derek-thomas/squad-v1.1-t5-question-generation"

To load the dataset, we use the `load_dataset()` method from the 🤗 Datasets library.


In [6]:
from datasets import load_dataset

# Load dataset from the hub
dataset = load_dataset(dataset_id)

print(f"Train dataset size: {len(dataset['train'])}")
print(f"Validation dataset size: {len(dataset['validation'])}")

# Train dataset size: 14732
# Test dataset size: 819

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/18896 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2067 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/derek-thomas___parquet/derek-thomas--squad-v1.1-t5-question-generation-5bdfd922a02702a2/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Train dataset size: 18896
Validation dataset size: 2067


Lets checkout an example of the dataset.

In [7]:
from random import randrange        


sample = dataset['train'][randrange(len(dataset["train"]))]
print(f"dialogue: \n{sample['context']}\n---------------")
print(f"summary: \n{sample['questions']}\n---------------")

dialogue: 
generate questions: Hastings was entrusted with the power of peace and war. British judges and magistrates would also be sent to India to administer the legal system. The Governor General and the council would have complete legislative powers. The company was allowed to maintain its virtual monopoly over trade in exchange for the biennial sum and was obligated to export a minimum quantity of goods yearly to Britain. The costs of administration were to be met by the company. The Company initially welcomed these provisions, but the annual burden of the payment contributed to the steady decline of its finances.
---------------
summary: 
in British indian a jugde had to come from where to oversee the  legal system? {sep_token} The highest ranking persons title in British india is? {sep_token} The highest ranking person's title in British india is? {sep_token} Did EIC have a monopoly over the governement or trade in british india {sep_token} Other then money what did it cost the 

To train our model we need to convert our inputs (text) to token IDs. This is done by a 🤗 Transformers Tokenizer. If you are not sure what this means check out [chapter 6](https://huggingface.co/course/chapter6/1?fw=tf) of the Hugging Face Course.

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="google/flan-t5-base"

# Load tokenizer of FLAN-t5-base
tokenizer = AutoTokenizer.from_pretrained(model_id)


Lets make sure we are handling the `{sep_token}` appropriately.

In [9]:
tokenizer.sep_token

Using sep_token, but it is not set yet.


In [10]:
tokenizer.sep_token = "{sep_token}"
tokenizer.add_tokens([tokenizer.sep_token])
print(tokenizer.sep_token, tokenizer.sep_token_id)

{sep_token} 32100


In [11]:
test_sequence = 'This {sep_token} is a {sep_token} test.'
print(tokenizer.tokenize(test_sequence))
tokenizer.encode(test_sequence)

['▁This', '▁', '{sep_token}', '▁is', '▁', 'a', '▁', '{sep_token}', '▁test', '.']


[100, 3, 32100, 19, 3, 9, 3, 32100, 794, 5, 1]

It looks like we are handling `{sep_token}` correctly.

Before we can start training we need to preprocess our data. Question Generation is a **text**2**text**-generation task. Meaning we wil take **text** (context) as an input and generate **text** (questions) as an output.

This data has been formatted as (AQPL) meaning all the questions are on one line. 

In [12]:
from datasets import concatenate_datasets

# The maximum total input sequence length after tokenization. 
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["validation"]]).map(lambda x: tokenizer(x["context"], truncation=True), batched=True, remove_columns=["context", "questions"])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization. 
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["validation"]]).map(lambda x: tokenizer(x["questions"], truncation=True), batched=True, remove_columns=["context", "questions"])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

  0%|          | 0/21 [00:00<?, ?ba/s]

Max source length: 512


  0%|          | 0/21 [00:00<?, ?ba/s]

Max target length: 413


In [13]:
def preprocess_function(sample,padding="max_length"):
#     # add prefix to the input for t5
    inputs = [item for item in sample["context"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["questions"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["context", "questions"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

  0%|          | 0/19 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


## 3. Fine-tune and evaluate FLAN-T5

After we have processed our dataset, we can start training our model. Therefore we first need to load our [FLAN-T5](https://huggingface.co/models?search=flan-t5) from the Hugging Face Hub. In the example we are using a instance with a NVIDIA V100 meaning that we will fine-tune the `base` version of the model. 
_I plan to do a follow-up post on how to fine-tune the `xxl` version of the model using Deepspeed._


In [14]:
from transformers import AutoModelForSeq2SeqLM

# huggingface hub model id
model_id="google/flan-t5-base"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id).to(device)

Since we added a token we need to resize the model. It can sometimes be a [speedup](https://twitter.com/karpathy/status/1607791537978748929?s=20&t=zL60wOBZ-V_2m5lDoDEocg) to make it a multiple of 64.

In [15]:
model.resize_token_embeddings(len(tokenizer)+(64-len(tokenizer)%64))

Embedding(32128, 768)

We want to evaluate our model during training. The `Trainer` supports evaluation during training by providing a `compute_metrics`.  
The most commonly used metrics to evaluate summarization task is [rogue_score](https://en.wikipedia.org/wiki/ROUGE_(metric)) short for Recall-Oriented Understudy for Gisting Evaluation). This metric does not behave like the standard accuracy: it will compare a generated summary against a set of reference summaries

We are going to use `evaluate` library to evaluate the `rogue` score.

In [16]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Before we can start training is to create a `DataCollator` that will take care of padding our inputs and labels. We will use the `DataCollatorForSeq2Seq` from the 🤗 Transformers library. 

In [17]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)


The last step is to define the hyperparameters (`TrainingArguments`) we want to use for our training. We are leveraging the [Hugging Face Hub](https://huggingface.co/models) integration of the `Trainer` to automatically push our checkpoints, logs and metrics during training into a repository.

In [27]:
repository_id

'flan-t5-base-derek-thomas/squad-v1.1-t5-question-generation'

In [36]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Hugging Face repository id
repository_id = f"{model_id.split('/')[1]}-{dataset_id}"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir="flan-t5-end2end-questions-generation",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=16,
    predict_with_generate=True,
    # fp16=False, # Overflows with fp16
    learning_rate=1e-4,
    num_train_epochs=10,
    # logging & evaluation strategies
    logging_dir=f"flan-t5-end2end-questions-generation/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_total_limit=4,
    load_best_model_at_end=True,
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    report_to="tensorboard",
    push_to_hub=True,
    hub_strategy="every_save",
    push_to_hub_model_id='flan-t5-end2end-questions-generation',
    hub_token=HfFolder.get_token(),
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics,
)

using `logging_steps` to initialize `eval_steps` to 500
PyTorch: setting up devices
/usr/local/lib/python3.8/dist-packages/transformers/training_args.py:1334: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case derek-thomas/flan-t5-end2end-questions-generation).
  warnings.warn(
/content/flan-t5-end2end-questions-generation is already a clone of https://huggingface.co/derek-thomas/flan-t5-end2end-questions-generation. Make sure you pull the latest changes with `repo.git_pull()`.


We can start our training by using the `train` method of the `Trainer`.

In [37]:
# Start training
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 18896
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 16
  Total optimization steps = 1470
  Number of trainable parameters = 247577856


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
500,1.149700,1.493632,30.347700,15.926600,25.458500,29.154700,18.993711
1000,1.267000,1.484685,30.351400,15.940800,25.488100,29.169200,18.998065


***** Running Evaluation *****
  Num examples = 2067
  Batch size = 8
Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_to

TrainOutput(global_step=1470, training_loss=1.2195334090667518, metrics={'train_runtime': 5167.3438, 'train_samples_per_second': 36.568, 'train_steps_per_second': 0.284, 'total_flos': 1.2933696873037824e+17, 'train_loss': 1.2195334090667518, 'epoch': 10.0})


![flan-t5-tensorboard](https://github.com/datavistics/flan-t5-e2e-qg/blob/assets/flan-t5-tensorboard.png?raw=1)

Nice, we have trained our model. 🎉 Lets run evaluate the best model again on the test set.


In [21]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 2067
  Batch size = 8
Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}



Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.0"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_

{'eval_loss': 1.4508943557739258,
 'eval_rouge1': 30.271,
 'eval_rouge2': 15.7962,
 'eval_rougeL': 25.3763,
 'eval_rougeLsum': 29.1224,
 'eval_gen_len': 19.0,
 'eval_runtime': 145.4846,
 'eval_samples_per_second': 14.208,
 'eval_steps_per_second': 1.78,
 'epoch': 5.0}

The best score we achieved is an `rouge1` score of `47.23`. 

Lets save our results and tokenizer to the Hugging Face Hub and create a model card. 

In [26]:
??trainer.push_to_hub

In [38]:
# Save our tokenizer and create model card
tokenizer.save_pretrained(repository_id)
trainer.create_model_card()
# Push the results to the hub
trainer.push_to_hub()

tokenizer config file saved in flan-t5-base-derek-thomas/squad-v1.1-t5-question-generation/tokenizer_config.json
Special tokens file saved in flan-t5-base-derek-thomas/squad-v1.1-t5-question-generation/special_tokens_map.json
Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}, 'metrics': [{'name': 'Rouge1', 'type': 'rouge', 'value': 30.3514}]}
Saving model checkpoint to flan-t5-end2end-questions-generation
Configuration saved in flan-t5-end2end-questions-generation/config.json
Configuration saved in flan-t5-end2end-questions-generation/generation_config.json
Model weights saved in flan-t5-end2end-questions-generation/pytorch_model.bin
remote: Scanning LFS files for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/derek-thomas/flan-t5-end2end-questions-generation
   6d57aaa..e541443  main -> main

remote: LFS file scan complete.       

'https://huggingface.co/derek-thomas/flan-t5-end2end-questions-generation/commit/e54144321e1e0c8764cafd2e930fd4d6af1bc22c'

## 4. Run Inference

Now we have a trained model, we can use it to run inference. We will use the `pipeline` API from transformers and a `test` example from our dataset.

In [ ]:
from transformers import pipeline
from random import randrange        

# load model and tokenizer from huggingface hub with pipeline
summarizer = pipeline("summarization", model="philschmid/flan-t5-base-samsum", device=0)

# select a random test sample
sample = dataset['test'][randrange(len(dataset["test"]))]
print(f"dialogue: \n{sample['dialogue']}\n---------------")

# summarize dialogue
res = summarizer(sample["dialogue"])

print(f"flan-t5-base summary:\n{res[0]['summary_text']}")

dialogue: 
Abby: Have you talked to Miro?
Dylan: No, not really, I've never had an opportunity
Brandon: me neither, but he seems a nice guy
Brenda: you met him yesterday at the party?
Abby: yes, he's so interesting
Abby: told me the story of his father coming from Albania to the US in the early 1990s
Dylan: really, I had no idea he is Albanian
Abby: he is, he speaks only Albanian with his parents
Dylan: fascinating, where does he come from in Albania?
Abby: from the seacoast
Abby: Duress I believe, he told me they are not from Tirana
Dylan: what else did he tell you?
Abby: That they left kind of illegally
Abby: it was a big mess and extreme poverty everywhere
Abby: then suddenly the border was open and they just left 
Abby: people were boarding available ships, whatever, just to get out of there
Abby: he showed me some pictures, like <file_photo>
Dylan: insane
Abby: yes, and his father was among the people
Dylan: scary but interesting
Abby: very!
---------------
flan-t5-base summary:
A